In [ ]:
import os, sys
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from databaseHelper import *
from sheetHelper import *
from datetime import timedelta
import psycopg2
from datetime import datetime
from openpyxl import Workbook

In [ ]:
dfTickets = adbFetch(""" 
    SELECT
        month,
        date,
        complainant_id as driverId,
        category_name,
        issue_name,
        ticket_status as status,
        ticket_id as ticketId
    FROM ticketModelNew 
    WHERE 
        date>='20250101'
        AND source IN ('Inbound', 'driverApp', 'Whatsapp')
        AND complainant_type = 'driver'
""")

dfTickets

In [ ]:
df_qc_failed = dfTickets[dfTickets['status'] == 'qc_failed']
df_others = dfTickets[dfTickets['status'] != 'qc_failed']

df_qc_failed

In [ ]:
df_others

In [ ]:
df_pairs = pd.merge(
    df_qc_failed,
    dfTickets,
    on=['driverId', 'category_name', 'issue_name'],
    suffixes=('_initial', '_other')
)

df_pairs = df_pairs[df_pairs['date_initial'] <= df_pairs['date_other']]
df_pairs = df_pairs[df_pairs['ticketId_initial'] != df_pairs['ticketId_other']]
df_pairsQC = df_pairs[df_pairs['status_other'] == 'qc_failed']
df_pairsOther = df_pairs[df_pairs['status_other'] != 'qc_failed']

df_pairs

In [ ]:
df_pairsQC

In [ ]:
df_pairsOther

In [ ]:
df_qc_failed_counts = df_qc_failed.groupby('month').size().reset_index(name='qc_failed_count')

df_pairsQC_counts = df_pairsQC.groupby('month_other').size().reset_index(name='followup_qc_failed_count')
df_pairsQC_counts.rename(columns={'month_other': 'month'}, inplace=True)

df_pairsOther_counts = df_pairsOther.groupby('month_other').size().reset_index(name='followup_other_status_count')
df_pairsOther_counts.rename(columns={'month_other': 'month'}, inplace=True)

final_df = pd.merge(df_qc_failed_counts, df_pairsQC_counts, on='month', how='outer')
final_df = pd.merge(final_df, df_pairsOther_counts, on='month', how='outer')

final_df = final_df.fillna(0)

final_df[['qc_failed_count', 'followup_qc_failed_count', 'followup_other_status_count']] = final_df[['qc_failed_count', 'followup_qc_failed_count', 'followup_other_status_count']].astype(int)

final_df = final_df.sort_values(by='month').reset_index(drop=True)

final_df

In [ ]:
write("https://docs.google.com/spreadsheets/d/12o-20N6qaZNisLvYD6jkjkACSaYuE-S9U_4x26Fl4ns/edit?gid=0#gid=0", "Sheet1", final_df)